# Model Loading

In [1]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    housing.data, housing.target, random_state = 42
)

# Model Building

In [3]:
import tensorflow as tf 
from tensorflow import keras

In [4]:
class My_Model(tf.keras.Model):
  def __init__(self, units=30, activation="relu"):
    super().__init__()
    self.norm_layer_wide = tf.keras.layers.Normalization()
    self.norm_layer_deep = tf.keras.layers.Normalization()
    self.hidden1 = tf.keras.layers.Dense(units, activation=activation)
    self.hidden2 = tf.keras.layers.Dense(units, activation=activation)
    self.main_output = tf.keras.layers.Dense(1)
  def call(self, inputs):
    input_wide, input_deep = inputs
    norm_wide = self.norm_layer_wide(input_wide)
    norm_deep = self.norm_layer_deep(input_deep)
    hidden1 = self.hidden1(norm_deep)
    hidden2 = self.hidden2(hidden1)
    concat = tf.keras.layers.concatenate([norm_wide, hidden2])
    output = self.main_output(concat)
    return output

In [5]:
tf.random.set_seed(42)
model = My_Model(30, activation="relu")

In [6]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(
    loss = "mse",
    optimizer=optimizer,
    metrics = ["RootMeanSquaredError"]
)

In [7]:
X_train_wide, X_train_deep = X_train[:,:5], X_train[:,2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:,2:]

In [8]:
model.norm_layer_wide.adapt(X_train_wide)
model.norm_layer_deep.adapt(X_train_deep)

# Model Training

In [9]:
history = model.fit((X_train_wide, X_train_deep),
                    y_train,
                    validation_split=0.1,
                    epochs = 10)

Epoch 1/10
436/436 [==============================] - 3s 3ms/step - loss: 1.6177 - root_mean_squared_error: 1.2719 - val_loss: 0.7565 - val_root_mean_squared_error: 0.8698
Epoch 2/10
436/436 [==============================] - 1s 3ms/step - loss: 0.5975 - root_mean_squared_error: 0.7730 - val_loss: 0.5613 - val_root_mean_squared_error: 0.7492
Epoch 3/10
436/436 [==============================] - 2s 4ms/step - loss: 0.4757 - root_mean_squared_error: 0.6897 - val_loss: 0.4721 - val_root_mean_squared_error: 0.6871
Epoch 4/10
436/436 [==============================] - 1s 3ms/step - loss: 0.4390 - root_mean_squared_error: 0.6626 - val_loss: 0.4627 - val_root_mean_squared_error: 0.6802
Epoch 5/10
436/436 [==============================] - 1s 2ms/step - loss: 0.4025 - root_mean_squared_error: 0.6345 - val_loss: 0.4151 - val_root_mean_squared_error: 0.6443
Epoch 6/10
436/436 [==============================] - 1s 3ms/step - loss: 0.3789 - root_mean_squared_error: 0.6155 - val_loss: 0.3955 - val_

# Model Evaluation

In [10]:
eval_results = model.evaluate((X_test_wide, X_test_deep), y_test)

162/162 [==============================] - 0s 3ms/step - loss: 0.3496 - root_mean_squared_error: 0.5913


In [11]:
eval_results

[0.34964966773986816, 0.5913118124008179]

# Prediction

In [12]:
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]

In [13]:
y_pred = model.predict((X_new_wide, X_new_deep))
y_pred 

1/1 [==============================] - 0s 192ms/step


array([[0.36844897],
       [1.4869882 ],
       [3.3377788 ]], dtype=float32)

In [14]:
y_test[:3]

array([0.477  , 0.458  , 5.00001])